# CLUSTERING - RANDOM FOREST

# Imports

In [25]:
import pandas as pd
import pickle
import numpy as np
from sklearn.externals import joblib
from sklearn import *
from sklearn import cluster
from sklearn.metrics import mean_squared_error,mean_absolute_error
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing, cross_validation
import warnings
warnings.filterwarnings('ignore')

## Load clean loan csv 

In [4]:
loan_df = pd.read_csv('/Users/sonalichaudhari/Desktop/Loan.csv', low_memory=False) 
del loan_df['Unnamed: 0']

In [5]:
loan_df.columns

Index(['id', 'debt_settlement_flag', 'application_type', 'fico_range_low',
       'fico_range_high', 'emp_length', 'dti', 'annual_inc', 'grade',
       'sub_grade', 'int_rate', 'loan_amnt', 'issue_d', 'purpose', 'State',
       'home_ownership', 'zip_code', 'policy_code', 'term', 'Year', 'Month',
       'fico', 'approval'],
      dtype='object')

In [6]:
loan_df['policy_code'] = loan_df['policy_code'].astype(int)

# Remove General Columns Not Needed

In [38]:
feature_df = loan_df[['application_type','emp_length', 'dti', 'annual_inc',
       'int_rate', 'loan_amnt', 'purpose', 'State', 'home_ownership', 'term', 'fico', 'approval']]

# Dealing with catagorical variables

In [9]:
home_ownerships = pd.get_dummies(feature_df['home_ownership'], prefix='house')
feature_df = feature_df.join(home_ownerships)
feature_df.drop('home_ownership', axis=1, inplace=True)

app_type = pd.get_dummies(feature_df['application_type'], prefix='appType')
feature_df = feature_df.join(app_type)
feature_df.drop('application_type', axis=1, inplace=True)

adr_states = pd.get_dummies(feature_df['State'], prefix='s')
feature_df = feature_df.join(adr_states)
feature_df.drop('State', axis=1, inplace=True)

#debt_settlement_flag  
debt_settle_flg = pd.get_dummies(feature_df['debt_settlement_flag'], prefix='stlFlg')
feature_df = feature_df.join(debt_settle_flg)
feature_df.drop('debt_settlement_flag', axis=1, inplace=True)     

#purpose   
purpose = pd.get_dummies(feature_df['purpose'], prefix='purpose')
feature_df = feature_df.join(purpose)
feature_df.drop('purpose', axis=1, inplace=True)

# No Clustering

In [10]:
zero_cluster_df = feature_df

In [11]:
X_train, X_test, y_train, y_test = train_test_split(zero_cluster_df.ix[:, zero_cluster_df.columns != 'int_rate'], 
                                            zero_cluster_df['int_rate'], 
                                            test_size=0.30)

In [17]:
rf = RandomForestRegressor(n_estimators=60)#max_depth =6,random_state=0)
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=60, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [18]:
# Accuracy of the model created
accuracy = rf.score(X_train, y_train)
print(accuracy)

0.9158103281


In [19]:
y_test_predicted = rf.predict(X_test)
y_train_predicted= rf.predict(X_train)

In [20]:
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100print
("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

-----Train-----
RMS:  1.3653576413677306
MAE:  1.02956180885
MAPE:  8.39008537442189
-----Test-----
RMS:  3.6157727578370595
MAE:  2.76234479412
MAPE:  8.39008537442189


# Manual Clustering

In [22]:
manual_df = feature_df
def clustering(fico):
    cluster_name = ''
    if fico>790.0:
        cluster_name ='K1'
    elif ((fico <= 790.0) & (fico>750.0)):
        cluster_name = 'K2'
    elif ((fico <=750.0) & (fico>700.0)):
        cluster_name ='K3'
    elif ((fico <=700.0) & (fico>660.0)):
        cluster_name= 'K4'
    return cluster_name
manual_df['cluster'] = manual_df['fico'].astype(float).map(lambda x: clustering(x))

In [23]:
manual_df.head()

,emp_length,dti,annual_inc,int_rate,loan_amnt,term,fico,approval,house_ANY,house_MORTGAGE,...,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,cluster
0,10,27.65,24000.0,10.65,5000.0,36,737.0,1,0,0,...,0,0,0,0,0,0,0,0,0,K3
1,1,1.00,30000.0,15.27,2500.0,60,742.0,1,0,0,...,0,0,0,0,0,0,0,0,0,K3
2,10,8.72,12252.0,15.96,2400.0,36,737.0,1,0,0,...,0,0,0,0,0,0,1,0,0,K3
3,10,20.00,49200.0,13.49,10000.0,36,692.0,1,0,0,...,0,0,0,0,1,0,0,0,0,K4
4,1,17.94,80000.0,12.69,3000.0,60,697.0,1,0,0,...,0,0,0,0,1,0,0,0,0,K4


In [26]:
#Seperate into diff dataframes
manual_c1 = manual_df[manual_df['cluster']=='K1']
manual_c1.drop('cluster',axis=1,inplace=True)
manual_c2 = manual_df[manual_df['cluster']=='K2']
manual_c2.drop('cluster',axis=1,inplace=True)
manual_c3 = manual_df[manual_df['cluster']=='K3']
manual_c3.drop('cluster',axis=1,inplace=True)
manual_c4 = manual_df[manual_df['cluster']=='K4']
manual_c4.drop('cluster',axis=1,inplace=True)

In [32]:
#Manual 1
X_train, X_test, y_train, y_test = train_test_split(manual_c1.ix[:, manual_c1.columns != 'int_rate'], 
                                            manual_c1['int_rate'], 
                                            test_size=0.30)
rf = RandomForestRegressor(n_estimators=60)#max_depth =6,random_state=0)
rf.fit(X_train, y_train)

# Accuracy of the model created
accuracy = rf.score(X_train, y_train)
print(accuracy)

y_test_predicted = rf.predict(X_test)
y_train_predicted= rf.predict(X_train)

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.912500708014
-----Train-----
RMS:  0.9272504824609101
MAE:  0.632214384474
MAPE:  7.502982856450695
-----Test-----
RMS:  2.5490267288809436
MAE:  1.73045444353
MAPE:  20.331065149803614


In [33]:
#manual 2
X_train, X_test, y_train, y_test = train_test_split(manual_c2.ix[:, manual_c2.columns != 'int_rate'], 
                                            manual_c2['int_rate'], 
                                            test_size=0.30)
rf = RandomForestRegressor(n_estimators=60)#max_depth =6,random_state=0)
rf.fit(X_train, y_train)

y_test_predicted = rf.predict(X_test)
y_train_predicted= rf.predict(X_train)

# Accuracy of the model created
accuracy = rf.score(X_train, y_train)
print(accuracy)



#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.906278490354
-----Train-----
RMS:  1.1164585319551865
MAE:  0.787638460991
MAPE:  8.58870617074232
-----Test-----
RMS:  2.9401312677168545
MAE:  2.10222240015
MAPE:  22.953272156575824


In [34]:
#Manual 3
X_train, X_test, y_train, y_test = train_test_split(manual_c3.ix[:, manual_c3.columns != 'int_rate'], 
                                            manual_c3['int_rate'], 
                                            test_size=0.30)
rf = RandomForestRegressor(n_estimators=60)#max_depth =6,random_state=0)
rf.fit(X_train, y_train)

# Accuracy of the model created
accuracy = rf.score(X_train, y_train)
print(accuracy)

y_test_predicted = rf.predict(X_test)
y_train_predicted= rf.predict(X_train)

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.906524048661
-----Train-----
RMS:  1.3058613344987609
MAE:  0.980578535877
MAPE:  8.977474505023931
-----Test-----
RMS:  3.452941052649615
MAE:  2.6302518139
MAPE:  24.109449922815536


In [35]:
#Manual 4
X_train, X_test, y_train, y_test = train_test_split(manual_c4.ix[:, manual_c4.columns != 'int_rate'], 
                                            manual_c4['int_rate'], 
                                            test_size=0.30)
rf = RandomForestRegressor(n_estimators=60)#max_depth =6,random_state=0)
rf.fit(X_train, y_train)

# Accuracy of the model created
accuracy = rf.score(X_train, y_train)
print(accuracy)

y_test_predicted = rf.predict(X_test)
y_train_predicted= rf.predict(X_train)

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.899470096252
-----Train-----
RMS:  1.4271983762261027
MAE:  1.08943011537
MAPE:  8.106339020466628
-----Test-----
RMS:  3.7778662018819844
MAE:  2.92132096202
MAPE:  21.797629211090126


# K-Means

In [39]:
k_cluster = joblib.load(open('/Users/sonalichaudhari/Desktop/ADS_A_2/Part 2/Clustering/kmeanCluster.pkl', 'rb'))
#Create new column in load_df for the k-means clustering 
k_mean_feature_df = feature_df
k_mean_feature_df['kMean'] = k_cluster.labels_

ValueError: Length of values does not match length of index

In [69]:
kmean_0 = k_mean_feature_df[k_mean_feature_df['kMean']==0]
kmean_0.drop('kMean', axis=1, inplace=True)
kmean_1 = k_mean_feature_df[k_mean_feature_df['kMean']==1]
kmean_1.drop('kMean', axis=1, inplace=True)
kmean_2 = k_mean_feature_df[k_mean_feature_df['kMean']==2]
kmean_2.drop('kMean', axis=1, inplace=True)
kmean_3 = k_mean_feature_df[k_mean_feature_df['kMean']==3]
kmean_3.drop('kMean', axis=1, inplace=True)
kmean_4 = k_mean_feature_df[k_mean_feature_df['kMean']==4]
kmean_4.drop('kMean', axis=1, inplace=True)
kmean_5 = k_mean_feature_df[k_mean_feature_df['kMean']==5]
kmean_5.drop('kMean', axis=1, inplace=True)
kmean_6 = k_mean_feature_df[k_mean_feature_df['kMean']==6]
kmean_6.drop('kMean', axis=1, inplace=True)
kmean_7 = k_mean_feature_df[k_mean_feature_df['kMean']==7]
kmean_7.drop('kMean', axis=1, inplace=True)


C:\Users\madhu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\madhu\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\madhu\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\madhu\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is tryi

In [71]:
#K-mean_0
X_train, X_test, y_train, y_test = train_test_split(kmean_0.ix[:, kmean_0.columns != 'int_rate'], 
                                            kmean_0['int_rate'], 
                                            test_size=0.30)
rf = RandomForestRegressor(n_estimators=60)#max_depth =6,random_state=0)
rf.fit(X_train, y_train)

# Accuracy of the model created
accuracy = rf.score(X_train, y_train)
print(accuracy)

y_test_predicted = rf.predict(X_test)
y_train_predicted= rf.predict(X_train)

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print('-----K-MEANS-----')
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100print
("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

Train
3.481010477367646
2.66740794826
22.839135830740016
Test
3.5076632435325483
2.6773989836
22.839135830740016


In [72]:
#K-mean_1
X_train, X_test, y_train, y_test = train_test_split(kmean_1.ix[:, kmean_1.columns != 'int_rate'], 
                                            kmean_1['int_rate'], 
                                            test_size=0.30)
rf = RandomForestRegressor(n_estimators=60)#max_depth =6,random_state=0)
rf.fit(X_train, y_train)

# Accuracy of the model created
accuracy = rf.score(X_train, y_train)
print(accuracy)

y_test_predicted = rf.predict(X_test)
y_train_predicted= rf.predict(X_train)

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print('-----K-MEANS-----')
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

C:\Users\madhu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


Train
3.968396816498858
3.08125947257
22.48082853149386
Test
3.97724072778496
3.09102714293
22.48082853149386


In [73]:
#K-mean_2
X_train, X_test, y_train, y_test = train_test_split(kmean_2.ix[:, kmean_2.columns != 'int_rate'], 
                                            kmean_2['int_rate'], 
                                            test_size=0.30)
rf = RandomForestRegressor(n_estimators=60)#max_depth =6,random_state=0)
rf.fit(X_train, y_train)

# Accuracy of the model created
accuracy = rf.score(X_train, y_train)
print(accuracy)

y_test_predicted = rf.predict(X_test)
y_train_predicted= rf.predict(X_train)

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print('-----K-MEANS-----')
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

C:\Users\madhu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


Train
3.499703146852312
2.68411807325
24.39473120159186
Test
3.4832858597908274
2.67235200681
24.39473120159186


In [74]:
#K-mean_3
X_train, X_test, y_train, y_test = train_test_split(kmean_3.ix[:, kmean_3.columns != 'int_rate'], 
                                            kmean_3['int_rate'], 
                                            test_size=0.30)
rf = RandomForestRegressor(n_estimators=60)#max_depth =6,random_state=0)
rf.fit(X_train, y_train)

# Accuracy of the model created
accuracy = rf.score(X_train, y_train)
print(accuracy)

y_test_predicted = rf.predict(X_test)
y_train_predicted= rf.predict(X_train)

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print('-----K-MEANS-----')
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

C:\Users\madhu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


Train
3.897652345255111
3.0170466115
21.102961069777077
Test
3.8839152452996277
3.01302701985
21.102961069777077


In [75]:
#K-mean_4
X_train, X_test, y_train, y_test = train_test_split(kmean_4.ix[:, kmean_4.columns != 'int_rate'], 
                                            kmean_4['int_rate'], 
                                            test_size=0.30)
rf = RandomForestRegressor(n_estimators=60)#max_depth =6,random_state=0)
rf.fit(X_train, y_train)

# Accuracy of the model created
accuracy = rf.score(X_train, y_train)
print(accuracy)

y_test_predicted = rf.predict(X_test)
y_train_predicted= rf.predict(X_train)

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print('-----K-MEANS-----')
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

Train
3.94450538392921
2.91398609669
20.35268750842468
Test
4.155850567263405
3.26998710294
20.35268750842468


C:\Users\madhu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


In [76]:
#K-mean_5
X_train, X_test, y_train, y_test = train_test_split(kmean_5.ix[:, kmean_5.columns != 'int_rate'], 
                                            kmean_5['int_rate'], 
                                            test_size=0.30)
rf = RandomForestRegressor(n_estimators=60)#max_depth =6,random_state=0)
rf.fit(X_train, y_train)

# Accuracy of the model created
accuracy = rf.score(X_train, y_train)
print(accuracy)

y_test_predicted = rf.predict(X_test)
y_train_predicted= rf.predict(X_train)

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print('-----K-MEANS-----')
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

C:\Users\madhu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


Train
3.690494692991596
2.83736919279
22.57399524126617
Test
3.69008617837717
2.83304008398
22.57399524126617


In [77]:
#K-mean_6
X_train, X_test, y_train, y_test = train_test_split(kmean_6.ix[:, kmean_6.columns != 'int_rate'], 
                                            kmean_6['int_rate'], 
                                            test_size=0.30)
rf = RandomForestRegressor(n_estimators=60)#max_depth =6,random_state=0)
rf.fit(X_train, y_train)

# Accuracy of the model created
accuracy = rf.score(X_train, y_train)
print(accuracy)

y_test_predicted = rf.predict(X_test)
y_train_predicted= rf.predict(X_train)

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print('-----K-MEANS-----')
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

C:\Users\madhu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


Train
3.374445936108331
2.58269110656
23.39543446216107
Test
3.3550426481630176
2.56792636635
23.39543446216107


In [78]:
#K-mean_7
X_train, X_test, y_train, y_test = train_test_split(kmean_7.ix[:, kmean_7.columns != 'int_rate'], 
                                            kmean_7['int_rate'], 
                                            test_size=0.30)
rf = RandomForestRegressor(n_estimators=60)#max_depth =6,random_state=0)
rf.fit(X_train, y_train)

# Accuracy of the model created
accuracy = rf.score(X_train, y_train)
print(accuracy)

y_test_predicted = rf.predict(X_test)
y_train_predicted= rf.predict(X_train)

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print('-----K-MEANS-----')
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

C:\Users\madhu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


Train
4.303993667625199
3.27449945154
22.12354231231697
Test
4.328852277974883
3.3188296155
22.12354231231697
